# Point-to-Mesh Similarity Analysis

This notebook demonstrates how to:
1. Load two meshes
2. Specify a particular point on the source mesh
3. Color-map the target mesh based on similarity to that point

The analysis uses diffusion features + DINO features to compute semantic similarity between mesh vertices.

## 1. Import Dependencies

In [1]:
import torch
from diff3f import get_features_per_vertex
from time import time
from utils import convert_mesh_container_to_torch_mesh, cosine_similarity, double_plot, get_colors, generate_colors
from dataloaders.mesh_container import MeshContainer
from diffusion import init_pipe
from dino import init_dino
from functional_map import compute_surface_map
import importlib
import meshplot as mp
from point_to_mesh_similarity import run_point_similarity_analysis, point_similarity_colormap, visualize_point_similarity, run_multi_point_correspondence_analysis

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

/home/zixiliu/miniforge3/envs/diff3f/lib/python3.10/site-packages/accelerate/utils/torch_xla.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Using device: cuda


## 2. Initialize Models

This step loads the diffusion pipeline and DINO model for feature extraction.

In [2]:
print("Initializing diffusion pipeline...")
pipe = init_pipe(device)

print("Initializing DINO model...")
dino_model = init_dino(device)

print("Models initialized successfully!")

Initializing diffusion pipeline...


/home/zixiliu/miniforge3/envs/diff3f/lib/python3.10/site-packages/diffusers/configuration_utils.py:239: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a model, please use <class 'unet_2d_condition.UNet2DConditionModel'>.load_config(...) followed by <class 'unet_2d_condition.UNet2DConditionModel'>.from_config(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  deprecate("config-passed-as-path", "1.0.0", deprecation_message, standard_warn=False)


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'pipeline_controlnet_img2img.StableDiffusionControlNetImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


Initializing DINO model...


Using cache found in /home/zixiliu/.cache/torch/hub/facebookresearch_dinov2_main
/home/zixiliu/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/home/zixiliu/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/home/zixiliu/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")


Models initialized successfully!


## 3. Load Meshes

Load your source and target meshes. You can modify the paths below to use your own meshes.

In [3]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp2.obj"
target_mesh_path = "meshes/oakink_bottle_decomp2.obj"

source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)


print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

Source mesh: 11799 vertices, 23334 faces
Target mesh: 9041 vertices, 17850 faces


## 4. Preview Meshes

Let's visualize the meshes before analysis to get familiar with their structure.

In [4]:
if 1:
    # Preview the meshes side by side
    d = mp.subplot(source_mesh.vert, source_mesh.face, s=[2, 2, 0])
    mp.subplot(target_mesh.vert, target_mesh.face, s=[2, 2, 1], data=d)
    print("Left: Source mesh | Right: Target mesh")

Left: Source mesh | Right: Target mesh


## 5. Configure Analysis Parameters

Set up the parameters for the similarity analysis.

In [5]:
# Analysis parameters
prompt = "a beaker"  # Text prompt for feature extraction
num_views = 10  # Number of views for rendering (reduced for faster computation)

print(f"\nAnalysis Configuration:")
print(f"- Text prompt: '{prompt}'")
print(f"- Number of views: {num_views}")


Analysis Configuration:
- Text prompt: 'a beaker'
- Number of views: 10


## 6. Run Similarity Analysis

This is the main computation step. It will:
1. Extract features for both meshes using multiple camera views
2. Compute similarity between the specified source point and all target vertices
3. Generate the visualization

⚠️ **Note**: This step can take several minutes depending on mesh complexity and number of views.

In [6]:
if 0:
    import torch
    
    # Clear cache of unused memory
    torch.cuda.empty_cache()
    
    # Optionally, collect unused memory from Python garbage collector
    import gc
    gc.collect()
    torch.cuda.empty_cache()

In [7]:
import numpy as np
source_point_3d = np.array([0.01231679, 0.0029739, -0.00678797])

if 1:

    # source_mesh_path = "meshes/oakink_beaker_decomp.obj"
    # target_mesh_path = "meshes/oakink_mug_decomp.obj"
    
    # source_mesh = MeshContainer().load_from_file(source_mesh_path)
    # target_mesh = MeshContainer().load_from_file(target_mesh_path)

    # if 0:
    #     # Set desired maximum edge length for uniform triangle size
    #     max_edge_length = 0.01  # Adjust as needed
    #     # Remesh source mesh
    #     new_source_vert, new_source_face = subdivide_to_size(
    #         source_mesh.vert, source_mesh.face, max_edge=max_edge_length
    #     )
    #     source_mesh.vert = new_source_vert
    #     source_mesh.face = new_source_face
    #     # Remesh target mesh
    #     new_target_vert, new_target_face = subdivide_to_size(
    #         target_mesh.vert, target_mesh.face, max_edge=max_edge_length
    #     )
    #     target_mesh.vert = new_target_vert
    #     target_mesh.face = new_target_face

    
    # print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
    # print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")
    
    # Analysis parameters
    prompt = "a beaker"  # Text prompt for feature extraction
    num_views = 4  # Number of views for rendering (reduced for faster computation)

    
    # Run the complete analysis pipeline
    similarity_colors, raw_similarities, source_point_idx, closest_distance = run_point_similarity_analysis(
        source_mesh=source_mesh,
        target_mesh=target_mesh,
        source_point_3d=source_point_3d,
        device=device,
        pipe=pipe,
        dino_model=dino_model,
        prompt=prompt,
        num_views=num_views
    )
    
    print("\n✅ Analysis complete!")

Input 3D point: [ 0.01231679  0.0029739  -0.00678797]
Closest vertex: 10093 (distance: 0.0221)
Closest vertex coordinates: [ 0.03249347  0.01198952 -0.00654438]
Computing features for source mesh...
Rendering complete


/home/zixiliu/miniforge3/envs/diff3f/lib/python3.10/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4322.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:11<00:00,  2.78s/it]


Number of missing features:  2667
Copied features from nearest vertices
Time taken in mins:  0.1998427391052246
Computing features for target mesh...
Rendering complete


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:08<00:00,  2.21s/it]

Number of missing features:  2336
Copied features from nearest vertices
Time taken in mins:  0.15821497837702433


ℹ️  Using vertex highlighting (3D markers unavailable: __enter__)
Source point (red): vertex 10093
Target mesh: colored by similarity + MOST SIMILAR vertex 1751 in RED
Most similar vertex similarity: 0.4561
Similarity range: 0.000 to 1.000

✅ Analysis complete!


# Multi point analysis

In [8]:
import numpy as np
source_points_3d = np.array([[-0.00793887, -0.0099004, -0.02071472], 
                             [0.01191107, 0.00449564, -0.02732792], 
                             [0.01248061, 0.00253191, -0.05360645], 
                             [0.01231679, 0.0029739, -0.00678797]])


# Analysis parameters
prompt = "a beaker"  # Text prompt for feature extraction
num_views = 10  # Number of views for rendering (reduced for faster computation)


# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)

print("\n✅ Analysis complete!")

Finding correspondences for 4 source points...
Closest vertices to input 3D points:
  Point 0: [-0.00793887 -0.0099004  -0.02071472] -> Vertex 7884 (distance: 0.0246)
  Point 1: [ 0.01191107  0.00449564 -0.02732792] -> Vertex 9922 (distance: 0.0217)
  Point 2: [ 0.01248061  0.00253191 -0.05360645] -> Vertex 197 (distance: 0.0112)
  Point 3: [ 0.01231679  0.0029739  -0.00678797] -> Vertex 10093 (distance: 0.0221)
Computing features for source mesh...
Rendering complete


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:24<00:00,  2.48s/it]


Number of missing features:  982
Copied features from nearest vertices
Time taken in mins:  0.4434179663658142
Computing features for target mesh...
Rendering complete


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:22<00:00,  2.24s/it]

Number of missing features:  937
Copied features from nearest vertices
Time taken in mins:  0.3974198261896769
  Point 0: Source vertex 7884 -> Target vertex 6005 (similarity: 0.9155)
  Point 1: Source vertex 9922 -> Target vertex 3778 (similarity: 0.7852)
  Point 2: Source vertex 197 -> Target vertex 72 (similarity: 0.5200)
  Point 3: Source vertex 10093 -> Target vertex 1728 (similarity: 0.7690)
🎨 Visualizing 4 correspondences:
  Correspondence 1: Source vertex 7884 ↔ Target vertex 6005 (similarity: 0.9155)
    Color: RGB(1.00, 0.20, 0.20)
  Correspondence 2: Source vertex 9922 ↔ Target vertex 3778 (similarity: 0.7852)
    Color: RGB(0.45, 0.90, 0.00)
  Correspondence 3: Source vertex 197 ↔ Target vertex 72 (similarity: 0.5200)
    Color: RGB(0.20, 1.00, 1.00)
  Correspondence 4: Source vertex 10093 ↔ Target vertex 1728 (similarity: 0.7690)
    Color: RGB(0.45, 0.00, 0.90)
🎯 Creating combined mesh + 3D marker visualization...


✅ Combined mesh + 3D octahedron markers visualization created!
   Each correspondence pair has matching colored 3D octahedrons
   Source mesh: 11799 vertices + 24 marker vertices
   Target mesh: 9041 vertices + 24 marker vertices

📊 Correspondence Statistics:
  - Average similarity: 0.7474
  - Min similarity: 0.5200
  - Max similarity: 0.9155
  - Total correspondences: 4

✅ Analysis complete!


# A few more examples

In [9]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp.obj"
target_mesh_path = "meshes/oakink_mug_decomp.obj"

# source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)

print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

# Analysis parameters
prompt = "a beaker"  # Text prompt for feature extraction
num_views = 4  # Number of views for rendering (reduced for faster computation)

print(f"\nAnalysis Configuration:")
print(f"- Text prompt: '{prompt}'")
print(f"- Number of views: {num_views}")



# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)

print("\n✅ Analysis complete!")

Source mesh: 11799 vertices, 23334 faces
Target mesh: 11392 vertices, 22696 faces

Analysis Configuration:
- Text prompt: 'a beaker'
- Number of views: 4
Finding correspondences for 4 source points...
Closest vertices to input 3D points:
  Point 0: [-0.00793887 -0.0099004  -0.02071472] -> Vertex 7884 (distance: 0.0246)
  Point 1: [ 0.01191107  0.00449564 -0.02732792] -> Vertex 9922 (distance: 0.0217)
  Point 2: [ 0.01248061  0.00253191 -0.05360645] -> Vertex 197 (distance: 0.0112)
  Point 3: [ 0.01231679  0.0029739  -0.00678797] -> Vertex 10093 (distance: 0.0221)
Computing features for source mesh...
Rendering complete


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:09<00:00,  2.42s/it]


Number of missing features:  2667
Copied features from nearest vertices
Time taken in mins:  0.17537790536880493
Computing features for target mesh...
Rendering complete


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:09<00:00,  2.28s/it]

Number of missing features:  2353
Copied features from nearest vertices
Time taken in mins:  0.16537917455037435
  Point 0: Source vertex 7884 -> Target vertex 8776 (similarity: 0.7314)
  Point 1: Source vertex 9922 -> Target vertex 5488 (similarity: 0.5967)
  Point 2: Source vertex 197 -> Target vertex 21 (similarity: 0.3557)
  Point 3: Source vertex 10093 -> Target vertex 4883 (similarity: 0.4807)
🎨 Visualizing 4 correspondences:
  Correspondence 1: Source vertex 7884 ↔ Target vertex 8776 (similarity: 0.7314)
    Color: RGB(1.00, 0.20, 0.20)
  Correspondence 2: Source vertex 9922 ↔ Target vertex 5488 (similarity: 0.5967)
    Color: RGB(0.45, 0.90, 0.00)
  Correspondence 3: Source vertex 197 ↔ Target vertex 21 (similarity: 0.3557)
    Color: RGB(0.20, 1.00, 1.00)
  Correspondence 4: Source vertex 10093 ↔ Target vertex 4883 (similarity: 0.4807)
    Color: RGB(0.45, 0.00, 0.90)
🎯 Creating combined mesh + 3D marker visualization...


✅ Combined mesh + 3D octahedron markers visualization created!
   Each correspondence pair has matching colored 3D octahedrons
   Source mesh: 11799 vertices + 24 marker vertices
   Target mesh: 11392 vertices + 24 marker vertices

📊 Correspondence Statistics:
  - Average similarity: 0.5411
  - Min similarity: 0.3557
  - Max similarity: 0.7314
  - Total correspondences: 4

✅ Analysis complete!


In [10]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp.obj"
target_mesh_path = "meshes/oakink_mug_decomp.obj"

# source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)

print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

# Analysis parameters
prompt = "a beaker"  # Text prompt for feature extraction
num_views = 10  # Number of views for rendering (reduced for faster computation)

print(f"\nAnalysis Configuration:")
print(f"- Text prompt: '{prompt}'")
print(f"- Number of views: {num_views}")



# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)

print("\n✅ Analysis complete!")

Source mesh: 11799 vertices, 23334 faces
Target mesh: 11392 vertices, 22696 faces

Analysis Configuration:
- Text prompt: 'a beaker'
- Number of views: 10
Finding correspondences for 4 source points...
Closest vertices to input 3D points:
  Point 0: [-0.00793887 -0.0099004  -0.02071472] -> Vertex 7884 (distance: 0.0246)
  Point 1: [ 0.01191107  0.00449564 -0.02732792] -> Vertex 9922 (distance: 0.0217)
  Point 2: [ 0.01248061  0.00253191 -0.05360645] -> Vertex 197 (distance: 0.0112)
  Point 3: [ 0.01231679  0.0029739  -0.00678797] -> Vertex 10093 (distance: 0.0221)
Computing features for source mesh...
Rendering complete


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:24<00:00,  2.48s/it]


Number of missing features:  982
Copied features from nearest vertices
Time taken in mins:  0.44326613744099935
Computing features for target mesh...
Rendering complete


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:22<00:00,  2.28s/it]

Number of missing features:  1590
Copied features from nearest vertices
Time taken in mins:  0.4084542314211527
  Point 0: Source vertex 7884 -> Target vertex 9296 (similarity: 0.7969)
  Point 1: Source vertex 9922 -> Target vertex 3626 (similarity: 0.8301)
  Point 2: Source vertex 197 -> Target vertex 21 (similarity: 0.3557)
  Point 3: Source vertex 10093 -> Target vertex 3273 (similarity: 0.7949)
🎨 Visualizing 4 correspondences:
  Correspondence 1: Source vertex 7884 ↔ Target vertex 9296 (similarity: 0.7969)
    Color: RGB(1.00, 0.20, 0.20)
  Correspondence 2: Source vertex 9922 ↔ Target vertex 3626 (similarity: 0.8301)
    Color: RGB(0.45, 0.90, 0.00)
  Correspondence 3: Source vertex 197 ↔ Target vertex 21 (similarity: 0.3557)
    Color: RGB(0.20, 1.00, 1.00)
  Correspondence 4: Source vertex 10093 ↔ Target vertex 3273 (similarity: 0.7949)
    Color: RGB(0.45, 0.00, 0.90)
🎯 Creating combined mesh + 3D marker visualization...


✅ Combined mesh + 3D octahedron markers visualization created!
   Each correspondence pair has matching colored 3D octahedrons
   Source mesh: 11799 vertices + 24 marker vertices
   Target mesh: 11392 vertices + 24 marker vertices

📊 Correspondence Statistics:
  - Average similarity: 0.6944
  - Min similarity: 0.3557
  - Max similarity: 0.8301
  - Total correspondences: 4

✅ Analysis complete!


In [11]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp.obj"
target_mesh_path = "meshes/oakink_wineglass_decomp.obj"

# source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)

print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

# Analysis parameters
prompt = "a beaker"  # Text prompt for feature extraction
num_views = 10  # Number of views for rendering (reduced for faster computation)

print(f"\nAnalysis Configuration:")
print(f"- Text prompt: '{prompt}'")
print(f"- Number of views: {num_views}")



# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)

print("\n✅ Analysis complete!")

Source mesh: 11799 vertices, 23334 faces
Target mesh: 11591 vertices, 23046 faces

Analysis Configuration:
- Text prompt: 'a beaker'
- Number of views: 10
Finding correspondences for 4 source points...
Closest vertices to input 3D points:
  Point 0: [-0.00793887 -0.0099004  -0.02071472] -> Vertex 7884 (distance: 0.0246)
  Point 1: [ 0.01191107  0.00449564 -0.02732792] -> Vertex 9922 (distance: 0.0217)
  Point 2: [ 0.01248061  0.00253191 -0.05360645] -> Vertex 197 (distance: 0.0112)
  Point 3: [ 0.01231679  0.0029739  -0.00678797] -> Vertex 10093 (distance: 0.0221)
Computing features for source mesh...
Rendering complete


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:24<00:00,  2.50s/it]


Number of missing features:  982
Copied features from nearest vertices
Time taken in mins:  0.44616601467132566
Computing features for target mesh...
Rendering complete


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:25<00:00,  2.57s/it]

Number of missing features:  768
Copied features from nearest vertices
Time taken in mins:  0.4579790790875753
  Point 0: Source vertex 7884 -> Target vertex 9423 (similarity: 0.6318)
  Point 1: Source vertex 9922 -> Target vertex 5300 (similarity: 0.6318)
  Point 2: Source vertex 197 -> Target vertex 11380 (similarity: 0.5396)
  Point 3: Source vertex 10093 -> Target vertex 1195 (similarity: 0.6470)
🎨 Visualizing 4 correspondences:
  Correspondence 1: Source vertex 7884 ↔ Target vertex 9423 (similarity: 0.6318)
    Color: RGB(1.00, 0.20, 0.20)
  Correspondence 2: Source vertex 9922 ↔ Target vertex 5300 (similarity: 0.6318)
    Color: RGB(0.45, 0.90, 0.00)
  Correspondence 3: Source vertex 197 ↔ Target vertex 11380 (similarity: 0.5396)
    Color: RGB(0.20, 1.00, 1.00)
  Correspondence 4: Source vertex 10093 ↔ Target vertex 1195 (similarity: 0.6470)
    Color: RGB(0.45, 0.00, 0.90)
🎯 Creating combined mesh + 3D marker visualization...


✅ Combined mesh + 3D octahedron markers visualization created!
   Each correspondence pair has matching colored 3D octahedrons
   Source mesh: 11799 vertices + 24 marker vertices
   Target mesh: 11591 vertices + 24 marker vertices

📊 Correspondence Statistics:
  - Average similarity: 0.6125
  - Min similarity: 0.5396
  - Max similarity: 0.6470
  - Total correspondences: 4

✅ Analysis complete!


In [20]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp.obj"
target_mesh_path = "meshes/oakink_wineglass_decomp.obj"

# source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)

print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

# Analysis parameters
prompt = "a beaker"  # Text prompt for feature extraction
num_views = 6  # Number of views for rendering (reduced for faster computation)

print(f"\nAnalysis Configuration:")
print(f"- Text prompt: '{prompt}'")
print(f"- Number of views: {num_views}")



# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)

print("\n✅ Analysis complete!")

Source mesh: 11799 vertices, 23334 faces
Target mesh: 11591 vertices, 23046 faces

Analysis Configuration:
- Text prompt: 'a beaker'
- Number of views: 6
Finding correspondences for 4 source points...
Closest vertices to input 3D points:
  Point 0: [-0.00793887 -0.0099004  -0.02071472] -> Vertex 7884 (distance: 0.0246)
  Point 1: [ 0.01191107  0.00449564 -0.02732792] -> Vertex 9922 (distance: 0.0217)
  Point 2: [ 0.01248061  0.00253191 -0.05360645] -> Vertex 197 (distance: 0.0112)
  Point 3: [ 0.01231679  0.0029739  -0.00678797] -> Vertex 10093 (distance: 0.0221)
Computing features for source mesh...
Rendering complete


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:14<00:00,  2.45s/it]


Number of missing features:  2400
Copied features from nearest vertices
Time taken in mins:  0.2647365132967631
Computing features for target mesh...
Rendering complete


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:15<00:00,  2.55s/it]

Number of missing features:  2798
Copied features from nearest vertices
Time taken in mins:  0.2738132079442342
  Point 0: Source vertex 7884 -> Target vertex 7110 (similarity: 0.6230)
  Point 1: Source vertex 9922 -> Target vertex 1367 (similarity: 0.7212)
  Point 2: Source vertex 197 -> Target vertex 11354 (similarity: 0.5396)
  Point 3: Source vertex 10093 -> Target vertex 1115 (similarity: 0.6274)
🎨 Visualizing 4 correspondences:
  Correspondence 1: Source vertex 7884 ↔ Target vertex 7110 (similarity: 0.6230)
    Color: RGB(1.00, 0.20, 0.20)
  Correspondence 2: Source vertex 9922 ↔ Target vertex 1367 (similarity: 0.7212)
    Color: RGB(0.45, 0.90, 0.00)
  Correspondence 3: Source vertex 197 ↔ Target vertex 11354 (similarity: 0.5396)
    Color: RGB(0.20, 1.00, 1.00)
  Correspondence 4: Source vertex 10093 ↔ Target vertex 1115 (similarity: 0.6274)
    Color: RGB(0.45, 0.00, 0.90)
🎯 Creating combined mesh + 3D marker visualization...


✅ Combined mesh + 3D octahedron markers visualization created!
   Each correspondence pair has matching colored 3D octahedrons
   Source mesh: 11799 vertices + 24 marker vertices
   Target mesh: 11591 vertices + 24 marker vertices

📊 Correspondence Statistics:
  - Average similarity: 0.6278
  - Min similarity: 0.5396
  - Max similarity: 0.7212
  - Total correspondences: 4

✅ Analysis complete!


In [12]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp.obj"
target_mesh_path = "meshes/oakink_bowl_decomp.obj"

# source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)

print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

# Analysis parameters
prompt = "a beaker"  # Text prompt for feature extraction
num_views = 10  # Number of views for rendering (reduced for faster computation)

print(f"\nAnalysis Configuration:")
print(f"- Text prompt: '{prompt}'")
print(f"- Number of views: {num_views}")



# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)

print("\n✅ Analysis complete!")

Source mesh: 11799 vertices, 23334 faces
Target mesh: 15566 vertices, 30916 faces

Analysis Configuration:
- Text prompt: 'a beaker'
- Number of views: 10
Finding correspondences for 4 source points...
Closest vertices to input 3D points:
  Point 0: [-0.00793887 -0.0099004  -0.02071472] -> Vertex 7884 (distance: 0.0246)
  Point 1: [ 0.01191107  0.00449564 -0.02732792] -> Vertex 9922 (distance: 0.0217)
  Point 2: [ 0.01248061  0.00253191 -0.05360645] -> Vertex 197 (distance: 0.0112)
  Point 3: [ 0.01231679  0.0029739  -0.00678797] -> Vertex 10093 (distance: 0.0221)
Computing features for source mesh...
Rendering complete


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:24<00:00,  2.49s/it]


Number of missing features:  982
Copied features from nearest vertices
Time taken in mins:  0.4528127153714498
Computing features for target mesh...
Rendering complete


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:24<00:00,  2.43s/it]

Number of missing features:  1371
Copied features from nearest vertices
Time taken in mins:  0.4431464711825053
  Point 0: Source vertex 7884 -> Target vertex 3626 (similarity: 0.5425)
  Point 1: Source vertex 9922 -> Target vertex 9198 (similarity: 0.5269)
  Point 2: Source vertex 197 -> Target vertex 9448 (similarity: 0.5132)
  Point 3: Source vertex 10093 -> Target vertex 2610 (similarity: 0.5752)
🎨 Visualizing 4 correspondences:
  Correspondence 1: Source vertex 7884 ↔ Target vertex 3626 (similarity: 0.5425)
    Color: RGB(1.00, 0.20, 0.20)
  Correspondence 2: Source vertex 9922 ↔ Target vertex 9198 (similarity: 0.5269)
    Color: RGB(0.45, 0.90, 0.00)
  Correspondence 3: Source vertex 197 ↔ Target vertex 9448 (similarity: 0.5132)
    Color: RGB(0.20, 1.00, 1.00)
  Correspondence 4: Source vertex 10093 ↔ Target vertex 2610 (similarity: 0.5752)
    Color: RGB(0.45, 0.00, 0.90)
🎯 Creating combined mesh + 3D marker visualization...


✅ Combined mesh + 3D octahedron markers visualization created!
   Each correspondence pair has matching colored 3D octahedrons
   Source mesh: 11799 vertices + 24 marker vertices
   Target mesh: 15566 vertices + 24 marker vertices

📊 Correspondence Statistics:
  - Average similarity: 0.5394
  - Min similarity: 0.5132
  - Max similarity: 0.5752
  - Total correspondences: 4

✅ Analysis complete!


In [13]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp.obj"
target_mesh_path = "meshes/oakink_mug2_decomp.obj"

# source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)

print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

# Analysis parameters
prompt = "a beaker"  # Text prompt for feature extraction
num_views = 10  # Number of views for rendering (reduced for faster computation)

print(f"\nAnalysis Configuration:")
print(f"- Text prompt: '{prompt}'")
print(f"- Number of views: {num_views}")



# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)

print("\n✅ Analysis complete!")

Source mesh: 11799 vertices, 23334 faces
Target mesh: 23304 vertices, 45840 faces

Analysis Configuration:
- Text prompt: 'a beaker'
- Number of views: 10
Finding correspondences for 4 source points...
Closest vertices to input 3D points:
  Point 0: [-0.00793887 -0.0099004  -0.02071472] -> Vertex 7884 (distance: 0.0246)
  Point 1: [ 0.01191107  0.00449564 -0.02732792] -> Vertex 9922 (distance: 0.0217)
  Point 2: [ 0.01248061  0.00253191 -0.05360645] -> Vertex 197 (distance: 0.0112)
  Point 3: [ 0.01231679  0.0029739  -0.00678797] -> Vertex 10093 (distance: 0.0221)
Computing features for source mesh...
Rendering complete


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:29<00:00,  2.95s/it]


Number of missing features:  982
Copied features from nearest vertices
Time taken in mins:  0.5213716427485148
Computing features for target mesh...
Rendering complete


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:31<00:00,  3.19s/it]

Number of missing features:  3917
Copied features from nearest vertices
Time taken in mins:  0.5942243059476217
  Point 0: Source vertex 7884 -> Target vertex 8059 (similarity: 0.7974)
  Point 1: Source vertex 9922 -> Target vertex 9522 (similarity: 0.8232)
  Point 2: Source vertex 197 -> Target vertex 322 (similarity: 0.5249)
  Point 3: Source vertex 10093 -> Target vertex 4826 (similarity: 0.7974)
🎨 Visualizing 4 correspondences:
  Correspondence 1: Source vertex 7884 ↔ Target vertex 8059 (similarity: 0.7974)
    Color: RGB(1.00, 0.20, 0.20)
  Correspondence 2: Source vertex 9922 ↔ Target vertex 9522 (similarity: 0.8232)
    Color: RGB(0.45, 0.90, 0.00)
  Correspondence 3: Source vertex 197 ↔ Target vertex 322 (similarity: 0.5249)
    Color: RGB(0.20, 1.00, 1.00)
  Correspondence 4: Source vertex 10093 ↔ Target vertex 4826 (similarity: 0.7974)
    Color: RGB(0.45, 0.00, 0.90)
🎯 Creating combined mesh + 3D marker visualization...


✅ Combined mesh + 3D octahedron markers visualization created!
   Each correspondence pair has matching colored 3D octahedrons
   Source mesh: 11799 vertices + 24 marker vertices
   Target mesh: 23304 vertices + 24 marker vertices

📊 Correspondence Statistics:
  - Average similarity: 0.7357
  - Min similarity: 0.5249
  - Max similarity: 0.8232
  - Total correspondences: 4

✅ Analysis complete!


In [14]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp.obj"
target_mesh_path = "meshes/oakink_vase_decomp.obj"

# source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)

print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

# Analysis parameters
prompt = "a beaker"  # Text prompt for feature extraction
num_views = 10  # Number of views for rendering (reduced for faster computation)

print(f"\nAnalysis Configuration:")
print(f"- Text prompt: '{prompt}'")
print(f"- Number of views: {num_views}")



# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)

print("\n✅ Analysis complete!")

Source mesh: 11799 vertices, 23334 faces
Target mesh: 7845 vertices, 15610 faces

Analysis Configuration:
- Text prompt: 'a beaker'
- Number of views: 10
Finding correspondences for 4 source points...
Closest vertices to input 3D points:
  Point 0: [-0.00793887 -0.0099004  -0.02071472] -> Vertex 7884 (distance: 0.0246)
  Point 1: [ 0.01191107  0.00449564 -0.02732792] -> Vertex 9922 (distance: 0.0217)
  Point 2: [ 0.01248061  0.00253191 -0.05360645] -> Vertex 197 (distance: 0.0112)
  Point 3: [ 0.01231679  0.0029739  -0.00678797] -> Vertex 10093 (distance: 0.0221)
Computing features for source mesh...
Rendering complete


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:32<00:00,  3.28s/it]


Number of missing features:  982
Copied features from nearest vertices
Time taken in mins:  0.5796547253926595
Computing features for target mesh...
Rendering complete


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:29<00:00,  2.99s/it]

Number of missing features:  572
Copied features from nearest vertices
Time taken in mins:  0.5217287739117941
  Point 0: Source vertex 7884 -> Target vertex 1459 (similarity: 0.7407)
  Point 1: Source vertex 9922 -> Target vertex 4677 (similarity: 0.6577)
  Point 2: Source vertex 197 -> Target vertex 7254 (similarity: 0.3733)
  Point 3: Source vertex 10093 -> Target vertex 4648 (similarity: 0.6421)
🎨 Visualizing 4 correspondences:
  Correspondence 1: Source vertex 7884 ↔ Target vertex 1459 (similarity: 0.7407)
    Color: RGB(1.00, 0.20, 0.20)
  Correspondence 2: Source vertex 9922 ↔ Target vertex 4677 (similarity: 0.6577)
    Color: RGB(0.45, 0.90, 0.00)
  Correspondence 3: Source vertex 197 ↔ Target vertex 7254 (similarity: 0.3733)
    Color: RGB(0.20, 1.00, 1.00)
  Correspondence 4: Source vertex 10093 ↔ Target vertex 4648 (similarity: 0.6421)
    Color: RGB(0.45, 0.00, 0.90)
🎯 Creating combined mesh + 3D marker visualization...


✅ Combined mesh + 3D octahedron markers visualization created!
   Each correspondence pair has matching colored 3D octahedrons
   Source mesh: 11799 vertices + 24 marker vertices
   Target mesh: 7845 vertices + 24 marker vertices

📊 Correspondence Statistics:
  - Average similarity: 0.6035
  - Min similarity: 0.3733
  - Max similarity: 0.7407
  - Total correspondences: 4

✅ Analysis complete!


In [15]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp.obj"
target_mesh_path = "meshes/oakink_jar_decomp.obj"

# source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)

print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

# Analysis parameters
prompt = "a beaker"  # Text prompt for feature extraction
num_views = 10  # Number of views for rendering (reduced for faster computation)

print(f"\nAnalysis Configuration:")
print(f"- Text prompt: '{prompt}'")
print(f"- Number of views: {num_views}")



# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)

print("\n✅ Analysis complete!")

Source mesh: 11799 vertices, 23334 faces
Target mesh: 13513 vertices, 26746 faces

Analysis Configuration:
- Text prompt: 'a beaker'
- Number of views: 10
Finding correspondences for 4 source points...
Closest vertices to input 3D points:
  Point 0: [-0.00793887 -0.0099004  -0.02071472] -> Vertex 7884 (distance: 0.0246)
  Point 1: [ 0.01191107  0.00449564 -0.02732792] -> Vertex 9922 (distance: 0.0217)
  Point 2: [ 0.01248061  0.00253191 -0.05360645] -> Vertex 197 (distance: 0.0112)
  Point 3: [ 0.01231679  0.0029739  -0.00678797] -> Vertex 10093 (distance: 0.0221)
Computing features for source mesh...
Rendering complete


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:31<00:00,  3.10s/it]


Number of missing features:  982
Copied features from nearest vertices
Time taken in mins:  0.5497798164685567
Computing features for target mesh...
Rendering complete


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:30<00:00,  3.03s/it]

Number of missing features:  1574
Copied features from nearest vertices
Time taken in mins:  0.540040123462677
  Point 0: Source vertex 7884 -> Target vertex 5488 (similarity: 0.7646)
  Point 1: Source vertex 9922 -> Target vertex 7697 (similarity: 0.7485)
  Point 2: Source vertex 197 -> Target vertex 6349 (similarity: 0.3638)
  Point 3: Source vertex 10093 -> Target vertex 7696 (similarity: 0.7407)
🎨 Visualizing 4 correspondences:
  Correspondence 1: Source vertex 7884 ↔ Target vertex 5488 (similarity: 0.7646)
    Color: RGB(1.00, 0.20, 0.20)
  Correspondence 2: Source vertex 9922 ↔ Target vertex 7697 (similarity: 0.7485)
    Color: RGB(0.45, 0.90, 0.00)
  Correspondence 3: Source vertex 197 ↔ Target vertex 6349 (similarity: 0.3638)
    Color: RGB(0.20, 1.00, 1.00)
  Correspondence 4: Source vertex 10093 ↔ Target vertex 7696 (similarity: 0.7407)
    Color: RGB(0.45, 0.00, 0.90)
🎯 Creating combined mesh + 3D marker visualization...


✅ Combined mesh + 3D octahedron markers visualization created!
   Each correspondence pair has matching colored 3D octahedrons
   Source mesh: 11799 vertices + 24 marker vertices
   Target mesh: 13513 vertices + 24 marker vertices

📊 Correspondence Statistics:
  - Average similarity: 0.6544
  - Min similarity: 0.3638
  - Max similarity: 0.7646
  - Total correspondences: 4

✅ Analysis complete!


## 7. Analyze Results

Let's examine the similarity scores and find the most similar regions.

In [ ]:
# Find the most similar points on target mesh
top_k = 10
most_similar_indices = np.argsort(raw_similarities)[-top_k:][::-1]
least_similar_indices = np.argsort(raw_similarities)[:top_k]

print(f"📊 Similarity Statistics:")
print(f"- Min similarity: {raw_similarities.min():.4f}")
print(f"- Max similarity: {raw_similarities.max():.4f}")
print(f"- Mean similarity: {raw_similarities.mean():.4f}")
print(f"- Std similarity: {raw_similarities.std():.4f}")

print(f"\n🔥 Top {top_k} most similar vertices on target mesh:")
for i, idx in enumerate(most_similar_indices):
    print(f"  {i+1:2d}. Vertex {idx:5d}: similarity = {raw_similarities[idx]:.4f}")

print(f"\n❄️  Bottom {top_k} least similar vertices on target mesh:")
for i, idx in enumerate(least_similar_indices):
    print(f"  {i+1:2d}. Vertex {idx:5d}: similarity = {raw_similarities[idx]:.4f}")

## 8. Visualize Similarity Distribution

Plot the distribution of similarity scores to understand the data better.

In [ ]:
import matplotlib.pyplot as plt 

# Plot similarity distribution
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(raw_similarities, bins=50, alpha=0.7, edgecolor='black')
plt.axvline(raw_similarities.mean(), color='red', linestyle='--', label=f'Mean: {raw_similarities.mean():.3f}')
plt.axvline(raw_similarities.max(), color='green', linestyle='--', label=f'Max: {raw_similarities.max():.3f}')
plt.xlabel('Similarity Score')
plt.ylabel('Frequency')
plt.title('Distribution of Similarity Scores')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(sorted(raw_similarities, reverse=True), linewidth=2)
plt.xlabel('Vertex Rank (sorted by similarity)')
plt.ylabel('Similarity Score')
plt.title('Similarity Scores (Ranked)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Interactive Visualization

The main result visualization showing:
- **Left**: Source mesh (gray) with the selected point highlighted in red
- **Right**: Target mesh colored by similarity (warmer colors = more similar)

You can interact with the 3D visualization to rotate and zoom the meshes.

In [ ]:
print("🎨 Interactive Visualization:")
print("- Source mesh (left): Gray with red point showing the reference vertex")
print("- Target mesh (right): Colored by similarity (warmer colors = more similar)")
print("- Use mouse to rotate and zoom the 3D view")

# The visualization was already created by run_point_similarity_analysis
# But let's create it again with custom colormap options
visualize_point_similarity(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_point_idx=source_point_idx,
    similarity_colors=similarity_colors,
)

## 10. Experiment with Different Points

Try analyzing different points on the source mesh to see how similarity patterns change.

In [ ]:
# Experiment with different source points
# You can change these indices and re-run this cell
experiment_points = [100, 500, 1500, 2000]  # Different vertex indices to try

for point_idx in experiment_points:
    if point_idx < len(source_mesh.vert):
        print(f"\n🔬 Experimenting with source vertex {point_idx}...")

        # Quick analysis with fewer views for faster computation
        exp_colors, exp_similarities = point_similarity_colormap(
            device=device,
            pipe=pipe,
            dino_model=dino_model,
            source_mesh=source_mesh,
            target_mesh=target_mesh,
            source_point_idx=point_idx,
            prompt=prompt,
            num_views=10  # Fewer views for quick experiment
        )

        # Show quick stats
        top_match = np.argmax(exp_similarities)
        print(f"  Best match: vertex {top_match} (similarity: {exp_similarities[top_match]:.4f})")

        # Visualize
        visualize_point_similarity(
            source_mesh=source_mesh,
            target_mesh=target_mesh,
            source_point_idx=point_idx,
            similarity_colors=exp_colors,
            colormap='viridis'
        )
    else:
        print(f"Skipping vertex {point_idx} (mesh only has {len(source_mesh.vert)} vertices)")

## 11. Save Results (Optional)

Save the similarity data for later analysis.

In [ ]:
# Save results to files
output_prefix = f"similarity_results_vertex_{source_point_idx}"

# Save similarity scores
np.save(f"{output_prefix}_similarities.npy", raw_similarities)
np.save(f"{output_prefix}_colors.npy", similarity_colors)

# Save analysis summary
summary = {
    'source_mesh_path': source_mesh_path,
    'target_mesh_path': target_mesh_path,
    'source_point_idx': source_point_idx,
    'prompt': prompt,
    'num_views': num_views,
    'min_similarity': float(raw_similarities.min()),
    'max_similarity': float(raw_similarities.max()),
    'mean_similarity': float(raw_similarities.mean()),
    'std_similarity': float(raw_similarities.std()),
    'top_matches': [int(idx) for idx in most_similar_indices[:5]]
}

import json
with open(f"{output_prefix}_summary.json", 'w') as f:
    json.dump(summary, f, indent=2)

print(f"✅ Results saved:")
print(f"  - Similarities: {output_prefix}_similarities.npy")
print(f"  - Colors: {output_prefix}_colors.npy")
print(f"  - Summary: {output_prefix}_summary.json")

## Summary

This notebook demonstrated how to:

1. ✅ Load and visualize 3D meshes
2. ✅ Extract semantic features using diffusion + DINO models
3. ✅ Compute similarity between a specific source point and all target vertices
4. ✅ Visualize results with interactive 3D plots
5. ✅ Analyze similarity distributions and find best matches
6. ✅ Experiment with different source points
7. ✅ Save results for future analysis

### Next Steps

- Try different mesh pairs
- Experiment with different prompts
- Adjust the number of views for quality vs. speed trade-offs
- Use the saved similarity data for further analysis or applications